In [ ]:
import gradio as gr
import asyncio
from agent.chat_openai_factory import ChatOpenAIFactory
from agent.google_place_api import GooglePlaceAPI
from agent.trip_agent import TripAgent
from dotenv import load_dotenv
import os

async def get_trip_categories(city: str):
    load_dotenv()
    chat_factory = ChatOpenAIFactory(openai_api_key=os.getenv("OPENAI_API_KEY"))
    google_api = GooglePlaceAPI(api_key=os.getenv("GOOGLE_PLACE_API_KEY"))
    agent = TripAgent(chat_factory, google_api)
    
    categories = await agent.get_categories(city)
    return ", ".join(categories)

demo = gr.Interface(
    fn=lambda x: asyncio.run(get_trip_categories(x)),
    inputs=gr.Textbox(label="Enter a city name", placeholder="e.g., Tokyo"),
    outputs=gr.Textbox(label="Interesting Categories"),
    title="Trip Categories Generator",
    description="Get interesting activity categories for any city!"
)

demo.launch(share=True)

In [ ]:
load_dotenv()

async def get_trip_recommendations(city: str, n_recommendations: int, 
                                 people_count: int, budget: int, interests: str, progress=gr.Progress()):
    from dotenv import load_dotenv
    import os
    load_dotenv()
    chat_factory = ChatOpenAIFactory(openai_api_key=os.getenv("OPENAI_API_KEY"))
    google_api = GooglePlaceAPI(api_key=os.getenv("GOOGLE_PLACE_API_KEY"))
    agent = TripAgent(chat_factory, google_api)

    progress(0, desc="Starting...")
    preferences = TripPreference(
        location=city,
        people_count=people_count,
        budget=budget,
        interests=interests.split(','),
        trip_days=7
    )
    progress(0.1, desc="Getting recommendations...")
    recommendations = await agent.get_recommendations(n_recommendations, preferences)
    progress(0.9, desc="Getting recommendations...")
    output = ""
    for i, rec in enumerate(recommendations, 1):
        output += f"""
# 🏛️ {rec.name} 
 **📝 Agenda**: {rec.agenda} <br>
 **🎯 Category**: {rec.category} <br>
 **⭐ Rating**: {rec.rating}/5 <br>
 **🌐 Website**: {rec.website_uri} <br>
 **📍 Maps**: {rec.google_maps_place_link} <br>
 **💰 Estimated Cost**: ${rec.cost} <br>
 **⏰ Duration**: {rec.duration}h <br>
 **👥 Group Size**: {rec.peopleCount} <br>
 **📅 Best Time**: {rec.time} <br>
 **🕒 Hours**: {rec.regular_opening_hours} <br>

 ---

"""
    return output

demo = gr.Interface(
    fn=lambda city, n, people, budget, interests: asyncio.run(
        get_trip_recommendations(city, n, people, budget, interests)
    ),
    inputs=[
        gr.Textbox(label="City", placeholder="e.g., Tokyo"),
        gr.Number(label="Number of Recommendations", value=3, minimum=1),
        gr.Number(label="Group Size", value=2, minimum=1),
        gr.Number(label="Budget (USD)", value=1000),
        gr.Textbox(label="Interests (comma-separated)", placeholder="e.g., food, culture, history")
    ],
    outputs=gr.Markdown(label="Recommendations"),
    title="Trip Recommendations Generator",
    description="Get personalized attraction recommendations for your trip!"
)

demo.launch(share=True)

In [2]:
import gradio as gr
import asyncio
from agent.chat_openai_factory import ChatOpenAIFactory
from agent.google_place_api import GooglePlaceAPI
from agent.trip_agent import TripAgent
from dotenv import load_dotenv
from agent.trip_preference import TripPreference
import os
load_dotenv()
debug_it = ""
async def get_categories_fn(city: str):
    chat_factory = ChatOpenAIFactory(openai_api_key=os.getenv("OPENAI_API_KEY"))
    google_api = GooglePlaceAPI(api_key=os.getenv("GOOGLE_PLACE_API_KEY"))
    agent = TripAgent(chat_factory, google_api)
    categories = await agent.get_categories(city)
    return ", ".join(categories)

async def get_trip_recommendations(city: str, n_recommendations: int, 
                                 people_count: int, budget: int, interests: str,
                                 progress=gr.Progress()):
    chat_factory = ChatOpenAIFactory(openai_api_key=os.getenv("OPENAI_API_KEY"))
    google_api = GooglePlaceAPI(api_key=os.getenv("GOOGLE_PLACE_API_KEY"))
    agent = TripAgent(chat_factory, google_api)
    
    preferences = TripPreference(
        location=city,
        people_count=people_count,
        budget=budget,
        interests=interests.split(','),
        trip_days=7
    )
    
    recommendations = await agent.get_recommendations(n_recommendations, preferences)
    progress(0.5, desc=f"Found {len(recommendations)} recommendations")
    
    output = ""
    total_recs = len(recommendations)
    for i, rec in enumerate(recommendations, 1):
        progress((0.5 + (i/total_recs) * 0.5), desc=f"Formatting recommendation {i}/{total_recs}")
        output += f"""
# 🏛️ {i}. {rec.name} 
 **📝 Agenda**: {rec.agenda} <br>
 **🎯 Category**: {rec.category} <br>
 **⭐ Rating**: {rec.rating}/5 <br>
 **🌐 Website**: {rec.website_uri} <br>
 **📍 Maps**: {rec.google_maps_place_link} <br>
 **💰 Estimated Cost**: ${rec.cost} <br>
 **⏰ Duration**: {rec.duration}h <br>
 **👥 Group Size**: {rec.peopleCount} <br>
 **📅 Best Time**: {rec.time} <br>
 **🕒 Hours**: {rec.regular_opening_hours} <br>

 ---
"""
    return output
async def get_itinerary(recommendations_str: str, city: str, trip_days: int,
                       people_count: int, budget: int, progress=gr.Progress()):
    chat_factory = ChatOpenAIFactory(openai_api_key=os.getenv("OPENAI_API_KEY"))
    google_api = GooglePlaceAPI(api_key=os.getenv("GOOGLE_PLACE_API_KEY"))
    agent = TripAgent(chat_factory, google_api)
    
    preferences = TripPreference(
        location=city,
        trip_days=trip_days,
        people_count=people_count,
        budget=budget,
        interests=["all"]  # Default value as interests not needed for itinerary
    )   
    progress(0, desc="Generating itinerary...")
    output = await agent.get_itinerary_with_reflection(
        recommendations=recommendations_str,
        trip_preference=preferences
    )
    return output
    

with gr.Blocks(title="Trip Planner") as demo:
    gr.Markdown("# 🌍 Trip Planner")
    
    with gr.Tab("Categories"):
        city_input1 = gr.Textbox(label="City", placeholder="e.g., Tokyo")
        categories_button = gr.Button("Get Categories")
        categories_output = gr.Textbox(label="Interesting Categories")
        
    with gr.Tab("Recommendations"):
        with gr.Row():
            with gr.Column(scale=1):
                city_input2 = gr.Textbox(label="City", placeholder="e.g., Tokyo")
                n_recommendations = gr.Number(label="Number of Recommendations", value=3, minimum=1)
                people_count = gr.Number(label="Group Size", value=2, minimum=1)
                budget = gr.Number(label="Budget (USD)", value=1000)
                interests = gr.Textbox(label="Interests (comma-separated)", placeholder="e.g., food, culture, history")
                rec_button = gr.Button("Get Recommendations")
            with gr.Column(scale=2):
                recommendations_output = gr.Markdown(label="Recommendations")

    with gr.Tab("Itinerary Generator"):
        with gr.Row():
            with gr.Column(scale=1):
                city_input3 = gr.Textbox(label="City", placeholder="e.g., Tokyo")
                trip_days = gr.Number(label="Trip Duration (days)", value=3, minimum=1)
                people_count2 = gr.Number(label="Group Size", value=2, minimum=1)
                budget2 = gr.Number(label="Budget (USD)", value=1000)
                recommendations_input = gr.Textbox(label="Paste Recommendations", lines=10)
                itinerary_button = gr.Button("Generate Itinerary")
            
            with gr.Column(scale=2):
                itinerary_output = gr.Textbox(label="Itinerary")
    
    categories_button.click(
        fn=lambda x: asyncio.run(get_categories_fn(x)),
        inputs=city_input1,
        outputs=categories_output
    )
    
    rec_button.click(
        fn=lambda city, n, people, budget, interests: asyncio.run(
            get_trip_recommendations(city, n, people, budget, interests)
        ),
        inputs=[city_input2, n_recommendations, people_count, budget, interests],
        outputs=recommendations_output
    )

    itinerary_button.click(
        fn=lambda rec_str, city, days, people, budget: asyncio.run(
            get_itinerary(rec_str, city, days, people, budget)
        ),
        inputs=[recommendations_input, city_input3, trip_days, people_count2, budget2],
        outputs=itinerary_output
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://7ab691b88186440e6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
